In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

from __future__ import print_function
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils import plot_model

from keras.layers.recurrent import LSTM
from keras.callbacks import EarlyStopping
from keras.initializers import glorot_uniform
from keras.initializers import orthogonal
from keras.initializers import TruncatedNormal

Using TensorFlow backend.


In [0]:

# 学習データ
df1 = csv.reader(open('x-data.csv', 'r'))
data1 = [ v for v in df1]
mat = np.array(data1)
mat2 = mat[1:]                        # 見出し行を外す
x_data = mat2[:, 1:].astype(float)  # 2列目以降を抜き出してfloat変換
print('x_data.shape=', x_data.shape)

# ラベルデータ
# 1％以上／0％以上／-1％以上／-1％未満
df2 = csv.reader(open('t-data.csv', 'r'))
data2 = [ v for v in df2]
mat3 = np.array(data2)
mat4 = mat3[1:]                       # 見出し行を外す
t_data = mat4[:, 1:].astype(float)  # 2列目以降を抜き出してfloat変換
print('t_data.shape=', t_data.shape)

x_data.shape= (13292, 1)
t_data.shape= (13291, 4)


In [0]:
maxlen = 80              # 入力系列数
n_in = x_data.shape[1]   # 学習データ（＝入力）の列数
n_out = t_data.shape[1]  # ラベルデータ（=出力）の列数
len_seq = t_data.shape[0] - maxlen + 1
data = []
target = []
for i in range(0, len_seq):
    data.append(x_data[i:i+maxlen, :])
    target.append(t_data[i+maxlen-1, :])
    
    x = np.array(data).reshape(len(data), maxlen, n_in)
    t = np.array(target).reshape(len(data), n_out)
    
    #print(x.shape, t.shape)

# ここからソースコードの後半
n_train = int(len(data)*0.9)              # 訓練データ長
x_train,x_test = np.vsplit(x, [n_train])  # 学習データを訓練用とテスト用に分割
t_train,t_test = np.vsplit(t, [n_train])  # ラベルデータを訓練用とテスト用に分割

#print(x_train.shape, x_test.shape, t_train.shape, t_test.shape)



In [0]:
class Prediction :
  def __init__(self, maxlen, n_hidden, n_in, n_out):
    self.maxlen = maxlen
    self.n_hidden = n_hidden
    self.n_in = n_in
    self.n_out = n_out

  def create_model(self):
    model = Sequential()
    model.add(LSTM(self.n_hidden, batch_input_shape = (None, self.maxlen, self.n_in),
             kernel_initializer = glorot_uniform(seed=20181228), 
             recurrent_initializer = orthogonal(gain=1.0, seed=20181228), 
             dropout = 0.5, 
             recurrent_dropout = 0.5))
    model.add(Dropout(0.5))
    model.add(Dense(self.n_out, 
            kernel_initializer = glorot_uniform(seed=20181228)))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer = "RMSprop", metrics = ['categorical_accuracy'])
    return model

  # 学習
  def train(self, x_train, t_train, batch_size, epochs) :
    #early_stopping = EarlyStopping(patience=0, verbose=1)
    model = self.create_model()
    model.fit(x_train, t_train, batch_size = batch_size, epochs = epochs, verbose = 1,
          shuffle = True, validation_split = 0.1)
    return model

In [5]:
n_hidden = 80     # 出力次元
epochs = 100      # エポック数
batch_size = 10   # ミニバッチサイズ

# モデル定義
prediction = Prediction(maxlen, n_hidden, n_in, n_out)
# 学習
model = prediction.train(x_train, t_train, batch_size, epochs)
# テスト
score = model.evaluate(x_test, t_test, batch_size = batch_size, verbose = 1)
print("score:", score)

# 正答率、準正答率（騰落）集計
preds = model.predict(x_test)
correct = 0
semi_correct = 0
for i in range(len(preds)):
    pred = np.argmax(preds[i,:])
    tar = np.argmax(t_test[i,:])
    
    if pred == tar :
        correct += 1
    else :
        if pred+tar == 1 or pred+tar == 5 :
            semi_correct += 1

print("正答率:", 1.0 * correct / len(preds))
print("準正答率（騰落）:", 1.0 * (correct+semi_correct) / len(preds))

Train on 10701 samples, validate on 1189 samples
Epoch 1/100
10701/10701 [==============================] - 55s 5ms/step - loss: 1.2866 - categorical_accuracy: 0.3787 - val_loss: 1.4166 - val_categorical_accuracy: 0.2927
Epoch 2/100
10701/10701 [==============================] - 54s 5ms/step - loss: 1.2761 - categorical_accuracy: 0.3886 - val_loss: 1.4574 - val_categorical_accuracy: 0.2960
Epoch 3/100
10701/10701 [==============================] - 55s 5ms/step - loss: 1.2709 - categorical_accuracy: 0.3928 - val_loss: 1.4302 - val_categorical_accuracy: 0.3028
Epoch 4/100
10701/10701 [==============================] - 55s 5ms/step - loss: 1.2684 - categorical_accuracy: 0.3919 - val_loss: 1.4127 - val_categorical_accuracy: 0.2918
Epoch 5/100
10701/10701 [==============================] - 56s 5ms/step - loss: 1.2706 - categorical_accuracy: 0.3922 - val_loss: 1.4702 - val_categorical_accuracy: 0.3019
Epoch 6/100
10701/10701 [==============================] - 56s 5ms/step - loss: 1.2674 - ca